<a href="https://colab.research.google.com/github/ranjan2829/Quant/blob/main/Quant_%26_Algorithmic_trading_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unsupervised Learning on s&p 500 stocks


In [2]:
pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=ef6936791882016a065ce0089c119444831c0df1e67400209f02ae9bf8ef153e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

In [ ]:
df['garman_volatility'] = ((np.log(df['high']) - np.log(df['low']))**2) / 2 - (2 * np.log(2) - 1) * (np.log(df['close']) - np.log(df['open'])**2)


In [ ]:
df

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [ ]:

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [ ]:
df.xs('AAPL',level=1)['rsi'].plot()

In [ ]:
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


In [ ]:
df.xs('AAPL',level=1)['bb_low'].plot()

In [ ]:
df.xs("AAPL",level=1)['bb_high'].plot()
df.xs('AAPL',level=1)['bb_low'].plot()
df.xs("AAPL",level=1)['bb_mid'].plot()

In [ ]:
df

In [ ]:
def compute_atr(stock):
  atr=pandas_ta.atr(high=stock['high'],
                    low=stock['low'],
                    close=stock['close'],
                    length=14)
  return atr.sub(atr.mean()).div(atr.std())


In [ ]:
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [ ]:
df.xs('AAPL',level=1)['atr'].plot()

In [ ]:
def compute_macd(close):
  macd=pandas_ta.macd(close=close,length=20).iloc[:,0]
  return macd.sub(macd.mean()).div(macd.std())


In [ ]:
df['macd']=df.groupby(level=1,group_keys=False)['adj close'].apply(compute_macd)

In [ ]:
df.xs('AAPL',level=1)['macd'].plot()

In [ ]:
df['dollar_vol']=(df['adj close']*df['volume'])/1e6

In [ ]:
df.xs('AAPL',level=1)['dollar_vol'].plot()

In [ ]:
df

## 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.

* To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [ ]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open',
                                                          'high', 'low', 'close']]



In [ ]:
df

In [ ]:
df['dollar_vol'] = (df.loc[:, 'dollar_vol'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

df['dollar_vol_rank'] = (df.groupby('date')['dollar_vol'].rank(ascending=False))

df= df[df['dollar_vol_rank']<150].drop(['dollar_vol', 'dollar_vol_rank'], axis=1)



In [ ]:
df